In [1]:
# -- Pre-run: change to parent directory for importing library--only run this once!
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
os.chdir(module_path)

In [2]:
# -- Notebook start
import folium
import logging

from datakit import Itinerum
import datakit_config

logging.getLogger().setLevel(logging.WARNING)

In [3]:
# initialize Itinerum library with config
itinerum = Itinerum(config=datakit_config)

# read .csvs defined in config if not already available
itinerum.setup(force=False)

In [16]:
# load all users and find a real one with many coordinates recorded by ID
users = itinerum.load_all_users()

test_user = users[2]

{}

In [6]:
# run trip detection on user with config parameters
parameters = {
    'subway_stations': itinerum.database.load_subway_entrances(),
    'break_interval_seconds': 120,
    'subway_buffer_meters': datakit_config.TRIP_DETECTION_SUBWAY_BUFFER_METERS,
    'cold_start_distance': datakit_config.TRIP_DETECTION_COLD_START_DISTANCE_METERS,
    'accuracy_cutoff_meters': datakit_config.TRIP_DETECTION_ACCURACY_CUTOFF_METERS    
}
results = itinerum.run_process(itinerum.process.trip_detection.triplab.algorithm, [test_user], parameters)

In [7]:
# save detected trips for user
trips, summaries = results[test_user]
trip_points = []
for trip in trips.values():
    for point in trip:
        point['uuid'] = test_user.uuid
        trip_points.append(point)
itinerum.database.save_trips(trip_points)

# reload user to build trip objects (this should be done automatically by trip detection,
# perhaps there should be a wrapper in the .trip_detection module to perform this on all add algorithms)
test_user = itinerum.database.load_user(uuid=test_user.uuid)

# examine the user's trips
test_user.trips

{1: <Trip num=1 code=1>}

In [9]:
# examine a trip's attributes
test_trip = test_user.trips[1]
print('distance:', test_trip.distance)
print('start:', test_trip.start_UTC, 'end:', test_trip.end_UTC)
print('code:', test_trip.trip_code)
test_trip

distance: 108837.69448774507
start: 2018-04-12 16:03:51 end: 2018-04-12 16:09:36
code: 1


<Trip num=1 code=1>

In [ ]:
# plot trip on map
m = folium.Map()
coordinates = [(p.latitude, p.longitude) for p in test_trip.points]
polyline = folium.PolyLine(locations=coordinates).add_to(m)
m.fit_bounds(coordinates)
m

In [ ]:
# attempt to map match between various mode networks using OSRM and compare the match quality
itinerum.process.map_match.osrm.match(coordinates=test_trip.points, matcher='BIKING')

In [ ]:
# write geojson features
## write user coordinates to geojson as points
itinerum.io.write_input_geojson(cfg=datakit_config,
                                fn_base=str(test_user.uuid),
                                coordinates=test_user.coordinates,
                                prompts=test_user.prompt_responses,
                                cancelled_prompts=test_user.cancelled_prompt_responses)

## write user detected trip points and links as linestrings to geojson
itinerum.io.write_trips_geojson(cfg=datakit_config, fn_base=str(test_user.uuid), trips=test_user.trips)